In [53]:
import numpy as np
import itertools
from refnx._lib import flatten
from refnx.util import general

# Spatz distances
LS4 = 368.5
sample_distance = 6237
slit2_distance = 2327

In [51]:
def angler(angle, angles, slits=None):
    ss2hg = ss3hg = ss4hg = 0
    if slits is not None:
        ss2hg = slits[angle].get("ss2hg", 0)
        ss3hg = slits[angle].get("ss3hg", 0)
        ss4hg = slits[angle].get("ss4hg", 0)

    return f"omega_2theta {angles[angle]} {2 * angles[angle]} {ss2hg:.3f} {ss3hg:.3f} {ss4hg:.3f}"


def omega_2theta(omega, twotheta):
    return "omega_2theta 0 0 0 0 0"


def slit_optimiser(angles, footprint, resolution):
    slits = []
    for angle in angles:
        d1, d2 = general.slit_optimiser(
            footprint,
            resolution,
            angle,
            L12=L23,
            L2S=L3S,
            LS3=LS4,
            verbose=False,
        )
        d4 = general.height_of_beam_after_dx(d1, d2, L23, L3S + LS4)
        slits.append({"ss2hg": d1, "ss3hg": d2, "ss4hg": 1.2 * d4[1] + 2})
    return slits


def hslits(ss2hg, ss3hg, ss4hg):
    return f"hslits {ss2hg} {ss3hg} {ss4hg}"


def hslits_from_angler(angle, slits):
    return f"hslits {slits[angle]["ss2hg"]} {slits[angle]["ss3hg"]} {slits[angle]["ss4hg"]}"
    

def positioner(position, positions):
    offset_dct = positions[position]
    som_offset = offset_dct.get("som", 0)
    sxtop_offset = offset_dct.get("sxtop", 0)
    sx_offset = offset_dct.get("sx", 0)
    samplename = offset_dct.get("samplename", "")
    sc_offset = offset_dct.get("sc", 0)
    offsets = [f"drive sxtop {sxtop_offset} sx {sx_offset} sc {sc_offset}", f"rel som {som_offset}"]
    
    if len(samplename):
        offsets.insert(0, f"samplename {samplename}")
    return offsets


def setpos(motor, pos):
    return f"setpos {motor} {pos}"


def drive(motor, pos):
    return f"drive {motor} {pos}"


def acquire(time):
    return ["autosave 30", f"runscan dummy_motor 0 0 1 time {time} force true"]


def samplename(samplename):
    return f"samplename {samplename}"


def syringe(pump0_vol, pump0_rate, pump1_vol, pump1_rate):
    cmds = [
        f"hset /sample/syr1/pump0/rat {pump0_rate}",
        f"hset /sample/syr1/pump0/vol {pump0_vol}",
        f"hset /sample/syr1/pump1/rat {pump1_rate}",
        f"hset /sample/syr1/pump1/vol {pump1_vol}",
        "hset /sample/syr1/pump0/run run",
    ]
    return cmds


def hplc(v1, v2, v3, v4, rate=1.0, vol=1.0):
    # v1 + v2 + v3 + v4 must equal 100
    np.testing.assert_equal(v1 + v2 + v3 + v4, 100)
    cmds = [
        f"hset /sample/hplc/pump/ratio/setp {v1}/{v2}/{v3}/{v4}",
        f"hset /sample/hplc/pump/flow/setp {rate}",
        f"hset /sample/hplc/pump/volume/setp {vol}",
    ]
    return cmds


def temperature(temp):
    return f"hset /sample/tc1/loop1/setpoint {temp}"


def wait(time):
    return f"wait {time}"


def mvp(pos):
    if pos < 1 or pos > 6:
        raise ValueError("pos has to be an integer 1 <= pos <=6")
    return f"hset /sample/mvp1/Control/SetPoint {int(pos)}"

Example loop code:

```
set temps {15 45}
foreach temp $temps {
    hset sample/tc1/loop1/setpoint $temp
}
```

In [46]:
# alter ss3y
ss3y = 186.75  # mm
footprint = 50  # mm
resolution = 0.033  # mm


################################################
# don't alter
slit3_distance = sample_distance - (250 - ss3y)
L3S = sample_distance - slit3_distance
L23 = slit3_distance - slit2_distance
################################################

angles = [0.6, 3.2]
slits = slit_optimiser(angles, footprint, resolution)
positions = [
    {"sx": 0, "sc": 0, "som": 0.1, "sxtop": 0, "samplename": "thing"},  # 0
    {"sx": 0, "sc": 0, "som": 0, "sxtop": 0, "samplename": ""},  # 1
]

In [47]:
cmds = [
    mvp(1),
    syringe(2, 2, 0, 0),
    wait(120),
    angler(0, angles, slits),
    positioner(0, positions),
    acquire(3600),
    hslits(0, 0, 0),
    drive("detrot", 30),
    temperature(30)
]

In [48]:
print("\n".join(flatten(cmds)))

drive mvp_driveable 1
hset /sample/syr1/pump0/rat 2
hset /sample/syr1/pump0/vol 2
hset /sample/syr1/pump1/rat 0
hset /sample/syr1/pump1/vol 0
hset /sample/syr1/pump0/run run
wait 120
omega_2theta 0.6 1.2 1.894 0.484 2.902
samplename thing
drive sxtop 0 sx 0
rel som 0.1
autosave 30
runscan dummy motor 0 0 1 time 3600 force true
hslits 0 0 0
drive detrot 30
hset /sample/tc1/loop1/setpoint 30


In [85]:
a = np.linspace(0, 1, 11)
b = np.linspace(1, 0, 11)

def looper():
    al = a.tolist()
    bl = b.tolist()
    cmds = []
    cmds.append("set vola {" + f"{' '.join([str(v) for v in al])}" + "}")
    cmds.append("set volb {" + f"{' '.join([str(v) for v in bl])}" + "}")
    cmds.append foreach
    print(cmds)

In [86]:
looper()

['set vola {0.0 0.1 0.2 0.30000000000000004 0.4 0.5 0.6000000000000001 0.7000000000000001 0.8 0.9 1.0}', 'set volb {1.0 0.9 0.8 0.7 0.6 0.5 0.3999999999999999 0.29999999999999993 0.19999999999999996 0.09999999999999998 0.0}']
